The following code i will remove the overlap of coastal watershed with stream watershed

In [1]:
import arcpy
import os
import pandas as pd
import numpy as np
import geopandas as gpd

In [2]:
# Inputs
CW_path = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Coastalwetland\hitshoreline"
wetlands_avg_inun = os.path.join(CW_path, 'Wetland_connected_avg_inundation_NAD1983_shorelineinteraction_buffer2000m.shp')
wetlands_low_inun = os.path.join(CW_path, 'Wetland_connected_low_inundation_NAD1983_shorelineinteraction_buffer2000m.shp')
wetlands_high_inun = os.path.join(CW_path, 'Wetland_connected_high_inundation_NAD1983_shorelineinteraction_buffer2000m.shp')
wetlands_surge_inun = os.path.join(CW_path, 'Wetland_connected_surge_inundation_NAD1983_shorelineinteraction_buffer2000m.shp')


Streamwatershed_path = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Streamwatershed\PointWaterdhed_LH_NAD1983.shp"


# Outputs
outDb = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Coastalwetland\Removeoverlapstreamwatershed"
outWetland_avg = os.path.join(outDb, 'Wetland_connected_avg_inundation_NAD1983_shorelineinteraction_buffer2000m_noStreamOverlap.shp')
outWetland_low = os.path.join(outDb, 'Wetland_connected_low_inundation_NAD1983_shorelineinteraction_buffer2000m_noStreamOverlap.shp')
outWetland_high = os.path.join(outDb, 'Wetland_connected_high_inundation_NAD1983_shorelineinteraction_buffer2000m_noStreamOverlap.shp')
outWetland_surge = os.path.join(outDb, 'Wetland_connected_surge_inundation_NAD1983_shorelineinteraction_buffer2000m_noStreamOverlap.shp')

# parameters 
Coastal_num = "CW_Id"

In [3]:
# Add a column called Shape_Area to the wetlands_surge_inun shapefile and wetlands_surge_inun shapefile and calecate the area of each feature in meters squared
arcpy.AddField_management(wetlands_surge_inun, "Shape_Area", "DOUBLE")
arcpy.CalculateField_management(wetlands_surge_inun, "Shape_Area", "!SHAPE!.getArea('PLANAR', 'SQUAREMETERS')", "PYTHON3")

arcpy.AddField_management(wetlands_low_inun, "Shape_Area", "DOUBLE")
arcpy.CalculateField_management(wetlands_low_inun, "Shape_Area", "!SHAPE!.getArea('PLANAR', 'SQUAREMETERS')", "PYTHON3")



<Result 'D:\\Users\\abolmaal\\Arcgis\\NASAOceanProject\\GIS_layer\\Coastalwetland\\hitshoreline\\Wetland_connected_low_inundation_NAD1983_shorelineinteraction_buffer2000m.shp'>

In [6]:
# Add a column to the outWetland shapefiles called CW_Id that is equal to FID + 1
arcpy.management.AddField(wetlands_avg_inun, 'CW_Id', 'LONG')
arcpy.management.CalculateField(wetlands_avg_inun, 'CW_Id', '!FID! + 1', 'PYTHON3')
arcpy.management.AddField(wetlands_low_inun, 'CW_Id', 'LONG')   
arcpy.management.CalculateField(wetlands_low_inun, 'CW_Id', '!FID! + 1', 'PYTHON3')
arcpy.management.AddField(wetlands_high_inun, 'CW_Id', 'LONG')
arcpy.management.CalculateField(wetlands_high_inun, 'CW_Id', '!FID! + 1', 'PYTHON3')
arcpy.management.AddField(wetlands_surge_inun, 'CW_Id', 'LONG')
arcpy.management.CalculateField(wetlands_surge_inun, 'CW_Id', '!FID! + 1', 'PYTHON3')

<Result 'D:\\Users\\abolmaal\\Arcgis\\NASAOceanProject\\GIS_layer\\Coastalwetland\\hitshoreline\\Wetland_connected_surge_inundation_NAD1983_shorelineinteraction_buffer2000m.shp'>

In [8]:
# Ensure CW_Area exists on all wetlands shapefiles (handles missing Shape_Area on low/surge)
def ensure_area_field(shapefile):
    fields = [f.name for f in arcpy.ListFields(shapefile)]
    if 'CW_Area' in fields:
        return
    if 'Shape_Area' in fields:
        try:
            arcpy.management.AlterField(shapefile, 'Shape_Area', 'CW_Area', 'CW_Area')
            return
        except Exception:
            # if AlterField is not supported, fall back to creating CW_Area and recalculating
            pass
    arcpy.management.AddField(shapefile, 'CW_Area', 'DOUBLE')
    arcpy.management.CalculateGeometryAttributes(shapefile, [['CW_Area', 'AREA']], area_unit='SQUARE_METERS')

# apply to each input shapefile
for shp in [wetlands_avg_inun, wetlands_low_inun, wetlands_high_inun, wetlands_surge_inun]:
    ensure_area_field(shp)

# add a column name ID_Coastal that is equal to CW_Id
for shp in [wetlands_avg_inun, wetlands_low_inun, wetlands_high_inun, wetlands_surge_inun]:
    arcpy.management.AddField(shp, 'ID_Coastal', 'LONG')
    arcpy.management.CalculateField(shp, 'ID_Coastal', '!CW_Id!', 'PYTHON3')

# add centroid lat/lon columns
for shp in [wetlands_avg_inun, wetlands_low_inun, wetlands_high_inun, wetlands_surge_inun]:
    arcpy.management.AddField(shp, 'start_lat', 'DOUBLE')
    arcpy.management.AddField(shp, 'start_lon', 'DOUBLE')
    arcpy.management.CalculateField(shp, 'start_lat', '!SHAPE!.centroid.Y', 'PYTHON3')
    arcpy.management.CalculateField(shp, 'start_lon', '!SHAPE!.centroid.X', 'PYTHON3')


In [10]:
# remove the coastal wetlands that overlap with the stream watersheds
arcpy.analysis.Erase(wetlands_avg_inun, Streamwatershed_path, outWetland_avg)
arcpy.analysis.Erase(wetlands_low_inun, Streamwatershed_path, outWetland_low)
arcpy.analysis.Erase(wetlands_high_inun, Streamwatershed_path, outWetland_high)
arcpy.analysis.Erase(wetlands_surge_inun, Streamwatershed_path, outWetland_surge)
print("Finished removing overlap between coastal wetlands and stream watersheds.")

Finished removing overlap between coastal wetlands and stream watersheds.
